# Gemini API: A Story-driven Tutorial

Welcome to this story-driven tutorial for the Gemini API! This guide is designed for developers who are new to the Gemini API and want a fun, engaging way to learn about its capabilities. We'll go on a journey to create a children's story, and along the way, we'll explore various features of the Gemini API.

In this notebook, you will learn how to:
- Generate creative text for characters and stories.
- Use structured output (JSON mode) to organize your content.
- Ground your story in real-world information using web search.
- Use the model to analyze and describe video content from a YouTube link.
- Generate a title for your story.
- Create a cover for your book using the image generation model.

## Setup

### Install the Python SDK

In [ ]:
%pip install -q -U 'google-genai>=1.0.0' pytube

### Configure your API key

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GEMINI_API_KEY`.

In [ ]:
from google.colab import userdata
from google import genai
from google.genai import types
import pathlib
import time

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
client = genai.Client(api_key=GEMINI_API_KEY)

### Choose a model

In [ ]:
MODEL_ID = "gemini-1.5-pro-latest"

## Character and Story Creation

Let's start by creating the main characters for our story. We'll use a simple text prompt to ask the Gemini API to come up with a character.

In [ ]:
prompt = "Create a fun and adventurous character for a children's story. The character is an animal who loves to explore."
response = client.generate_content(model=MODEL_ID, contents=prompt)

print(response.text)

Now that we have our main character, let's create a sidekick for them.

In [ ]:
prompt = "Our main character is a curious and brave fox named Felix. Create a sidekick for Felix who is a bit clumsy but very loyal."
response = client.generate_content(model=MODEL_ID, contents=prompt)

print(response.text)

With our characters ready, let's generate a basic story outline.

In [ ]:
prompt = "Create a story outline about Felix the fox and his sidekick, a clumsy but loyal badger named Barnaby. Their adventure involves traveling to Paris for the Olympic games."
response = client.generate_content(model=MODEL_ID, contents=prompt)

print(response.text)

## Structured Output for Characters and Chapters

To better organize our story, we can use the JSON mode to get structured output from the Gemini API. This is useful when you need the model to return data in a specific format.

Let's start by defining our characters using a Pydantic model.

In [ ]:
from pydantic import BaseModel
import json

class Character(BaseModel):
    name: str
    species: str
    description: str

prompt = "Generate a list of JSON objects for the characters Felix the fox and Barnaby the badger, following the provided schema."

response = client.generate_content(
    model=MODEL_ID,
    contents=prompt,
    generation_config=types.GenerationConfig(
        response_mime_type="application/json",
        response_schema=[Character]
    )
)

print(response.text)

Now, let's create the chapters of our book in a structured format.

In [ ]:
class Chapter(BaseModel):
    chapter_number: int
    title: str
    summary: str

prompt = "Generate a list of chapters for a children's story about Felix the fox and Barnaby the badger traveling to the Paris Olympics. The story should have at least 5 chapters."

response = client.generate_content(
    model=MODEL_ID,
    contents=prompt,
    generation_config=types.GenerationConfig(
        response_mime_type="application/json",
        response_schema=[Chapter]
    )
)

print(response.text)

## Grounding with Web Search and Video

To make our story more engaging and educational, we can ground it in real-world information. The grounding feature allows the Gemini API to use Google Search to get up-to-date information about a topic.

In [ ]:
prompt = "Describe the opening ceremony of the Paris Olympic games. This will be for a children's story, so keep it simple and exciting."
response = client.generate_content(
    model=MODEL_ID,
    contents=prompt,
    tools=[types.Tool(google_search=types.GoogleSearch())]
)

print(response.text)

We can also use the Gemini API to analyze video content. Let's ask the model to describe a scene from the Paris Olympics opening ceremony using a YouTube link.

In [ ]:
from pytube import YouTube

youtube_url = "https://www.youtube.com/watch?v=xtlvuPxNKWI"
yt = YouTube(youtube_url)
stream = yt.streams.get_highest_resolution()
video_path = stream.download()

video_file = client.files.upload(path=video_path)

while video_file.state.name == "PROCESSING":
    print('Waiting for video to be processed.')
    time.sleep(10)
    video_file = client.files.get(name=video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.error.message)

prompt = "Watch this video and describe the scene at the 1:30 mark."
response = client.generate_content(model=MODEL_ID, contents=[prompt, video_file])

print(response.text)

## Generating a Title

Every good story needs a title! Let's ask Gemini to come up with a few options.

In [ ]:
title_model = "gemini-1.5-flash-latest"
prompt = "Suggest three titles for a children's story about a fox named Felix and a badger named Barnaby who go to the Paris Olympics."
response = client.generate_content(model=title_model, contents=prompt)

print(response.text)

## Generating a Book Cover

Now for the final touch: a book cover! We can use an image generation model to create a beautiful cover for our story.

In [ ]:
from IPython.display import Image, display

prompt = "Create a book cover for a children's story. The title is 'Felix and Barnaby's Parisian Adventure'. The cover should feature a friendly fox and a clumsy badger in Paris, with the Eiffel Tower and Olympic rings in the background. The style should be a colorful and whimsical illustration."

response = client.images.generate(prompt=prompt, model="imagen-3")

# Display the generated image
display(Image(url=response.images[0].url))

## Creating a Video (Placeholder)

While we can't generate a video directly in this notebook, you can use a text-to-video model like Veo to bring our story to life. You would provide the generated book cover image, and Veo could generate a video based on the image.

For example, you could take the generated book cover and use it as a prompt for Veo:

```
Prompt for Veo:

Animate this book cover. Make the clouds move, the characters blink, and the Olympic rings sparkle.
```

This would create a short animated video of the book cover!